## Introduction

We proposed 2 methods to generate clusters, using just orthograph or using also embeddings and semantics.

### Cluster Generator

In [6]:
import os
import re
from collections import defaultdict
from Levenshtein import distance
from typing import Dict, Set
import json

In [12]:
ORIGINAL_DATA_PATH = '../data/processed_chats'
ONLY_TEXT_DATA_PATH = '../data/only_text_chats'
CLUSTER_OUTPUT_PATH = 'normal_clusters.json'
SEMANTIC_CLUSTER_OUTPUT_PATH = 'semantic_clusters.json'

In [8]:
def extract_text_from_chats():
		# Create output diarectory if it doesn't exist
		os.makedirs(ONLY_TEXT_DATA_PATH, exist_ok=True)
		
		# Get all chat files
		chat_files = [f for f in os.listdir('../data/processed_chats') if f.endswith('.txt')]
		
		for chat_file in chat_files:
				input_path = os.path.join(ORIGINAL_DATA_PATH, chat_file)
				output_path = os.path.join(ONLY_TEXT_DATA_PATH, chat_file)
				
				with open(input_path, 'r', encoding='utf-8') as f:
						lines = f.readlines()
						
				# Extract only the text part after the timestamp and speaker
				text_lines = []
				for line in lines:
						# Match pattern: [timestamp] speaker: text
						match = re.match(r'^\[.*?\]\s+.*?:\s+(.+)$', line)
						if match:
								text = match.group(1).strip()
								if text:  # Only add non-empty messages
										text_lines.append(text + '\n')
				
				# Write only the text to new file
				with open(output_path, 'w', encoding='utf-8') as f:
						f.writelines(text_lines)
						
				print(f"Processed {chat_file}")

In [9]:
extract_text_from_chats()

Processed chat_1.txt
Processed chat_2.txt
Processed chat_3.txt
Processed chat_6.txt
Processed chat_4.txt


In [10]:
def cluster_similar_words(threshold: float = 0.8) -> Dict[str, Set[str]]:
    # Get all words from files
    words = set()
    chat_dir = '../data/only_text_chats'
    
    for filename in os.listdir(chat_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(chat_dir, filename), 'r', encoding='utf-8') as f:
                text = f.read().lower()
                # Extract words, keeping only alphanumeric characters
                file_words = re.findall(r'\b\w+\b', text)
                words.update(file_words)
    
    # Convert to list for easier indexing
    words = list(words)
    clusters = defaultdict(set)
    
    # Compare each word with every other word
    for i, word1 in enumerate(words):
        if word1 in clusters:  # Skip if already clustered
            continue
            
        clusters[word1].add(word1)  # Add word to its own cluster
        
        for word2 in words[i+1:]:
            if word2 in clusters:  # Skip if already clustered
                continue
                
            # Calculate similarity ratio
            max_len = max(len(word1), len(word2))
            if max_len == 0:
                continue
                
            similarity = 1 - (distance(word1, word2) / max_len)
            
            if similarity >= threshold:
                clusters[word1].add(word2)
                clusters[word2].add(word1)
    
    return clusters

In [ ]:
clusters = cluster_similar_words(threshold=0.8)

with open(CLUSTER_OUTPUT_PATH, 'w', encoding='utf-8') as f:
    json.dump({k: sorted(list(v)) for k, v in clusters.items() if len(v) >= 2}, f, ensure_ascii=False, indent=2)

### Semantic Cluster Generator

In [15]:
import os
import re
import json
import numpy as np
from collections import defaultdict
from Levenshtein import distance
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, Set
from tqdm import tqdm

/Users/joehachem/Desktop/ML_FINAL/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/joehachem/Desktop/ML_FINAL/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def cluster_semantic_words(spelling_threshold: float = 0.8, semantic_threshold: float = 0.7):
    """
    Cluster words based on both spelling and semantic similarity.
    
    Args:
        spelling_threshold: Minimum spelling similarity ratio (0-1)
        semantic_threshold: Minimum semantic similarity ratio (0-1)
    
    Returns:
        Dictionary mapping each word to its cluster of similar words
    """
    # Get all words from files
    print("Reading files and extracting words...")
    words = set()
    chat_dir = '../data/only_text_chats'
    
    for filename in os.listdir(chat_dir):
        if filename.endswith('.txt'):
            with open(os.path.join(chat_dir, filename), 'r', encoding='utf-8') as f:
                text = f.read().lower()
                file_words = re.findall(r'\b\w+\b', text)
                words.update(file_words)
    
    words = list(words)
    print(f"Found {len(words)} unique words")
    
    # Load multilingual model
    print("Loading multilingual model...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    
    # Get semantic embeddings
    print("Getting semantic embeddings...")
    embeddings = model.encode(words, show_progress_bar=True)
    
    # Calculate semantic similarity matrix
    print("Calculating semantic similarities...")
    semantic_sim = cosine_similarity(embeddings)
    
    # Initialize clusters
    clusters = defaultdict(set)
    
    # Compare each word with every other word
    print("Clustering words...")
    total_pairs = len(words) * (len(words) - 1) // 2
    processed_pairs = 0
    
    with tqdm(total=total_pairs, desc="Processing word pairs") as pbar:
        for i, word1 in enumerate(words):
            if word1 in clusters:  # Skip if already clustered
                pbar.update(len(words) - i - 1)
                continue
                
            clusters[word1].add(word1)  # Add word to its own cluster
            
            for j, word2 in enumerate(words[i+1:], i+1):
                if word2 in clusters:  # Skip if already clustered
                    continue
                    
                # Calculate spelling similarity
                max_len = max(len(word1), len(word2))
                if max_len == 0:
                    continue
                    
                spelling_sim = 1 - (distance(word1, word2) / max_len)
                
                # Get semantic similarity
                semantic_sim_ratio = semantic_sim[i, j]
                
                # Combine both similarities (you can adjust weights)
                combined_sim = (spelling_sim + semantic_sim_ratio) / 2
                
                if combined_sim >= semantic_threshold:
                    clusters[word1].add(word2)
                    clusters[word2].add(word1)
                
                processed_pairs += 1
                pbar.update(1)
    
    return clusters

In [ ]:
def save_clusters_to_json(clusters: Dict[str, Set[str]], min_cluster_size: int = 2):
    """Save clusters with at least min_cluster_size words to JSON"""
    # Convert sets to lists for JSON serialization
    filtered_clusters = {
        word: sorted(list(cluster))
        for word, cluster in clusters.items()
        if len(cluster) >= min_cluster_size
    }
    
    output_file = 'semantic_clusters.json'
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(filtered_clusters, f, ensure_ascii=False, indent=2)
    
    print(f"Saved {len(filtered_clusters)} clusters to {output_file}")

In [ ]:
clusters = cluster_semantic_words(spelling_threshold=0.8, semantic_threshold=0.7)
save_clusters_to_json(clusters, min_cluster_size=2)